In [ ]:
import csv
from pprint import pprint

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
plt.style.use('seaborn-whitegrid')

In [ ]:
!pip install psycopg2-binary

In [ ]:
import psycopg2
import psycopg2.extras as e

In [ ]:
dbname = 'solar_power'
user = 'postgres'
password = 'myshell123'
host = 'localhost'
port = '5432'

In [ ]:
#conn = psycopg2.connect("dbname=crappy_gifts user=postgres password=$kyw8lker host=localhost")
conn = psycopg2.connect(f"dbname={dbname} user={user} password={password} host={host}")
cursor = conn.cursor()
conn.dsn

## Solar Power Installations in New York State

In [ ]:
fp = "https://data.ny.gov/api/views/3x8r-34rs/rows.csv?accessType=DOWNLOAD&sorting=true"

df = pd.read_csv(fp, encoding='utf-8')

df.info()

In [ ]:
df.columns = df.columns.str.lower().str.replace(" ","_").str.replace("$","")
df.columns

In [ ]:
df.info()

In [ ]:
sql_columns = list(df.dtypes.index)
pprint(sql_columns, compact=True)

In [ ]:
date_cols = [x for x in df.columns if "date" in x.lower()]

df[date_cols] = df[date_cols].apply(lambda x: pd.to_datetime(x, format='%m/%d/%Y'))

In [ ]:
df['longitude'] = df['georeference'].str.extract('(-?\d+.\d+)\s', expand = True).astype('float32')
df['latitude'] = df['georeference'].str.extract('\s(-?\d+.\d+)', expand = True).astype('float32')

df[['georeference','latitude','longitude']].head()

In [ ]:
df.shape

In [ ]:
df = df[df['project_cost'].notna()]
df = df[df['incentive'].notna()]
df = df[df['county'].notna()]
df = df[df['electric_utility'].notna()]

df.shape

In [ ]:
df['total_costs'] = df.eval('project_cost + incentive')
df['total_costs'].sum()

In [ ]:
#df.dropna(how='any', inplace=True)

df = df[df['date_application_received'].between('2001-01-01','2022-01-01')]
df.shape

In [ ]:
table = "solar"

In [ ]:
fp = '{}.csv'.format(table)

df.to_csv(fp, sep='\t', index=False)

In [ ]:
pd_maps = {
    'object':'varchar',
    'datetime64[ns]':'date',
    'int64':'integer',
    'int32':'integer',
    'float64':'numeric',
    'float32':'numeric',
    'bool':'boolean'
}

In [ ]:
pd_columns = df.dtypes.index.tolist()
pprint(pd_columns, compact=True)

In [ ]:
pd_formats = df.dtypes.values.astype(str).tolist()
pprint(pd_formats, compact=True)

In [ ]:
cc = {}
for i, x in enumerate(pd_columns):
    cc[x] = pd_maps[pd_formats[i]]
    
cc

In [ ]:
cc['state'] = 'char(2)'

cc

In [ ]:
stmt = """DROP TABLE IF EXISTS {};""".format(table)
print(stmt)

In [ ]:
cursor.execute(stmt)
conn.commit()

In [ ]:
stmt = "CREATE TABLE {} (\n".format(table)

for k, v in cc.items():
    stmt += """"{}" {},\n""".format(k, v)

stmt = stmt[:-2]
stmt += "\n);"

print(stmt)

In [ ]:
cursor.execute(stmt)
conn.commit()

In [ ]:
stmt = """TRUNCATE TABLE {};""".format(table)
cursor.execute(stmt)
conn.commit()

In [ ]:
stmt = \
"""
copy {} from stdin
with csv header delimiter as e'\\t' null as ''
""".format(table)

print(stmt)

In [ ]:
with open(fp, 'r', encoding='utf-8') as f:
    cursor.copy_expert(sql=stmt, file=f)
    conn.commit()

In [ ]:
stmt = f'''
select *
from {table}
'''

data = pd.read_sql(con=conn, sql=stmt)
data.head(10)

## Counties of New York State

In [ ]:
url = "https://data.ny.gov/api/views/4xc7-bukh/rows.csv?accessType=DOWNLOAD&sorting=true"
print(url)

In [ ]:
ny_county = pd.read_csv(url)
ny_county

In [ ]:
ny_county.rename(columns={x: x.replace(" ", "_").lower()  for x in ny_county.columns}, inplace=True)

In [ ]:
set(df.county).symmetric_difference(set(ny_county.county))

In [ ]:
df.loc[df['county'] == 'St. Lawrence', ['county']] = 'St Lawrence'

In [ ]:
set(df.county).symmetric_difference(set(ny_county.county))

In [ ]:
ny_county['location'].head()

In [ ]:
del ny_county['location']

In [ ]:
del df

In [ ]:
df = ny_county.copy()

In [ ]:
pd_columns = df.dtypes.index.tolist()
pprint(pd_columns, compact=True)

In [ ]:
pd_formats = df.dtypes.values.astype(str).tolist()
pprint(pd_formats, compact=True)

In [ ]:
cc = {}
for i, x in enumerate(pd_columns):
    cc[x] = pd_maps[pd_formats[i]]
    
cc

In [ ]:
table = "counties"

In [ ]:
fp = '{}.csv'.format(table)

df.to_csv(fp, sep='\t', index=False)

In [ ]:
stmt = """DROP TABLE IF EXISTS {};""".format(table)
print(stmt)

In [ ]:
cursor.execute(stmt)
conn.commit()

In [ ]:
stmt = "CREATE TABLE {} (\n".format(table)

for k, v in cc.items():
    stmt += """"{}" {},\n""".format(k, v)

stmt = stmt[:-2]
stmt += "\n);"

print(stmt)

In [ ]:
cursor.execute(stmt)
conn.commit()

In [ ]:
stmt = """TRUNCATE TABLE {};""".format(table)
cursor.execute(stmt)
conn.commit()

In [ ]:
stmt = \
"""
copy {} from stdin
with csv header delimiter as e'\\t' null as ''
""".format(table)

print(stmt)

In [ ]:
with open(fp, 'r', encoding='utf-8') as f:
    cursor.copy_expert(sql=stmt, file=f)
    conn.commit()

In [ ]:
cursor.close()
conn.close()